# Outlier Identification and Removal

When modeling, it is important to clean the data sample to ensure that the observations best represent the problem. Sometimes a dataset can contain extreme values that are outside the range of what is expected and unlike the other data. These are called outliers and often machine learning modeling and model skill in general can be improved by understanding and even removing these outlier values.

In this exercise, you will discover outliers and how to identify and remove them from your machine learning dataset.

After completing this exercise, you will know:
- That an outlier is an unlikely observation in a dataset and may have one of many causes.
- How to use simple univariate statistics like standard deviation and interquartile range to identify and remove outliers from a data sample.
- How to use an outlier detection model to identify and remove rows from a training dataset in order to lift predictive modeling performance.

---

## 1. Exercise Overview

This exercise is divided into five parts; they are:
1. What are Outliers?
2. Test Dataset
3. Standard Deviation Method
4. Interquartile Range Method
5. Automatic Outlier Detection

---

## 2. What are Outliers?

An outlier is an observation that is unlike the other observations. They are rare, distinct, or do not fit in some way. We will generally define outliers as samples that are exceptionally far from the mainstream of the data.

Outliers can have many causes, such as:
- Measurement or input error.
- Data corruption.
- True outlier observation.

There is no precise way to defne and identify outliers in general because of the specifics of each dataset. Instead, you, or a domain expert, must interpret the raw observations and decide whether a value is an outlier or not.

Even with a thorough understanding of the data, outliers can be hard to define. Great care should be taken not to hastily remove or change values, especially if the sample size is small.

Nevertheless, we can use statistical methods to identify observations that appear to be rare or unlikely given the available data.

This does not mean that the values identified are outliers and should be removed. But, the tools described in this exercise can be helpful in shedding light on rare events that may require a second look. A good tip is to consider plotting the identified outlier values, perhaps in the context of non-outlier values to see if there are any systematic relationship or pattern to the outliers. If there is, perhaps they are not outliers and can be explained, or perhaps the outliers themselves can be identified more systematically.

---

## 3. Test Dataset

Before we look at outlier identification methods, let's define a dataset we can use to test the methods.

Numbers drawn from a Gaussian distribution will have outliers. That is, by virtue of the distribution itself, there will be a few values that will be a long way from the mean, rare values that we can identify as outliers.

- Using `randn()` from the `numpy.random` routine, generate a population 10,000 random numbers drawn from a Gaussian distribution with a mean of 50 and a standard deviation of 5. 

  Note: The `randn()` function generates a random Gaussian values with a mean of $0$ and a standard deviation of $1$. To get the preferred range, multiply the results by the standard deviation and add the mean to shift the values. You are to use a `seed(1)` before generating the random number (or rather pseudo-random) to ensure the same sample of numbers is generated each time the code is executed. 

- Determine the mean and standard deviation of the data you generated. You should expect your answer your values to be very close to 50 and 5 respectively. (Answer: mean=50.049 stdv=4.994)

---

## 4. Standard Deviation Method

If we know that the distribution of values in the sample is Gaussian or Gaussian-like, we can use the standard deviation of the sample as a cut-off for identifying outliers. The Gaussian distribution has the property that the standard deviation from the mean can be used to reliably summarize the percentage of values in the sample.

In statistics, within one standard deviation of the mean will cover 68 percent of the data. We can cover more of the data sample if we expand therange as follows:
- 1 Standard Deviation from the Mean: 68 percent.
- 2 Standard Deviations from the Mean: 95 percent.
- 3 Standard Deviations from the Mean: 99.7 percent.

This is known as the [empirical rule](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule). So, if the mean is 50 and the standard deviation is 5, as in the test dataset above, then all data in the sample between 45 and 55 will account for about 68 percent of the data sample.

A value that falls outside of 3 standard deviations is part of the distribution, but it is an unlikely or rare event at approximately 1 in 370 samples. Three standard deviations from the mean is a common cut-off in practice for identifying outliers in a Gaussian or Gaussian-like distribution. For smaller samples of data, perhaps a value of 2 standard deviations (95 percent) can be used, and for larger samples, perhaps a value of 4 standard deviations (99.9 percent) can be used.

Given the mean $\mu$ and standard deviation $\sigma$, a simple way to identify outliers is to compute a *z-score* for every $x_i$. *Z-score* is defined as the number of standard deviations away $x_i$ is from the mean. Data values that have a z-score sigma greater than a threshold, for example, of three, are declared to be outliers.

Let's make this concrete with a worked example. Sometimes, the data is standardized first (e.g. to a Z-score with zero mean and unit variance) so that the outlier detection can be performed using standard Z-score cut-off values. This is a convenience and is not required in general, and we will perform the calculations in the original scale of the data here to make things clear. We can calculate the mean and standard deviation of a given sample, then calculate the cut-off for identifying outliers as more than 3 standard deviations from the mean.
```python
'''Example of estimating the lower and upper bounds of the data.'''
...
# calculate summary statistics
data_mean, data_std = mean(data), std(data)
# define outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
```

We can then identify outliers as those examples that fall outside of the defined lower and upper limits.
```python
'''Example of identifying outliers using the limits on the data.'''
...
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
```

Alternately, we can filter out those values from the sample that are not within the defined limits.
```python
'''Example of removing outliers from the data.'''
...
# remove outliers
outliers_removed = [x for x in data if x > lower and x < upper]
```

- Using the methods described here, identify the number of outliers as compared to non-outlier observations from the data you have generated previously.
  
  Expected output
  ```
  Identified outliers: 29
  Non-outlier observations: 9971
  ```

So far we have only talked about univariate data with a Gaussian distribution, e.g. a single variable. You can use the same approach if you have multivariate data, e.g. data with multiple variables, each with a different Gaussian distribution. You can imagine bounds in two dimensions that would define an ellipse if you have two variables. Observations that fall outside of the ellipse would be considered outliers. In three dimensions, this would be an ellipsoid, and so on into higher dimensions. Alternately, if you knew more about the domain, perhaps an outlier may be identified by exceeding the limits on one or a subset of the data dimensions.

---

## 5. Interquartile Range Method

Not all data is normal or normal enough to treat it as being drawn from a Gaussian distribution. A good statistic for summarizing a non-Gaussian distribution sample of data is the Interquartile Range, or IQR for short. The IQR is calculated as the difference between the 75th and the 25th percentiles of the data and defines the box in a box and whisker plot. Remember that percentiles can be calculated by sorting the observations and selecting values at specific indices. The 50th percentile is the middle value, or the average of the two middle values for an even number of examples. If we had 10,000 samples, then the 50th percentile would be the average of the 5000th and 5001st values.

We refer to the percentiles as quartiles (quart meaning 4) because the data is divided into four groups via the 25th, 50th and 75th values. The IQR defines the middle 50 percent of the data, or the body of the data.

The IQR can be used to identify outliers by defining limits on the sample values that are a factor k of the IQR below the 25th percentile or above the 75th percentile. The common value for the factor $k$ is the value 1.5. A factor $k$ of 3 or more can be used to identify values that are extreme outliers or far outs when described in the context of box and whisker plots. On a box and whisker plot, these limits are drawn as fences on the whiskers (or the lines) that are drawn from the box. Values that fall outside of these values are drawn as dots. We can calculate the percentiles of a dataset using the `percentile()` NumPy function that takes the dataset and specification of the desired percentile. The IQR can then be calculated as the difference between the 75th and 25th percentiles.

```python
'''Example of calculating quartiles on the data.'''
...
# calculate interquartile range
q25, q75 = percentile(data, 25), percentile(data, 75)
iqr = q75 - q25
Listing 6.8: Example of calculating quartiles on the data.
```

We can then calculate the cutoff for outliers as 1.5 times the IQR and subtract this cut-off from the 25th percentile and add it to the 75th percentile to give the actual limits on the data.

```python
'''Example of calculating lower and upper bounds using the IQR.'''
...
# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
```

We can then use these limits to identify the outlier values.
```python
'''Example of identifying outliers using the limits on the data.'''
...
# identify outliers
outliers = [x for x in data if x < lower or x > upper]
```

We can also use the limits to filter out the outliers from the dataset.

```python
'''Example of removing outliers from the data.'''
...
# remove outliers
outliers_removed = [x for x in data if x > lower and x < upper]
```

- Using the methods described here, identify the number of outliers as compared to non-outlier observations from the data you have generated previously. Your code print 25th and 75th percentiles and the calculated IQR.
  
  Expected output
  ```
  Percentiles: 25th=46.685, 75th=53.359, IQR=6.674
  Identified outliers: 81
  Non-outlier observations: 9919
  ```

The approach can be used for multivariate data by calculating the limits on each variable in the dataset in turn, and taking outliers as observations that fall outside of the rectangle or hyper-rectangle.

---

## 6. Automatic Outlier Detection

In machine learning, an approach to tackling the problem of outlier detection is one-class classification. A simple approach to identifying outliers is to locate those examples that are far from the other examples in the multi-dimensional feature space. This can work well for feature spaces with low dimensionality (few features), although it can become less reliable as the number of features is increased, referred to as the curse of dimensionality. The local outlier factor, or `LOF` for short, is a technique that attempts to harness the idea of nearest neighbors for outlier detection. Each example is assigned a scoring of how isolated or how likely it is to be outliers based on the size of its local neighborhood. Those examples with the largest score are more likely to be outliers. The scikit-learn library provides an implementation of this approach in the `LocalOutlierFactor` class.

We can demonstrate the `LocalOutlierFactor` method on a predictive modeling dataset. 

We will use the Boston housing regression problem that has 13 inputs and one numerical target and requires learning the relationship between suburb characteristics and house prices.

- Using Pandas, load the dataset and peek at the top 5 values.

Looking in the dataset, you should see that all variables are numeric.

- Next, retrieve the data as a NumPy array using `to_numpy()` function.

- Then, separate it into input `X` and output `y` variables. They should have a shape of (506, 13) and (506,) respectively

- Using `train_test_split` from the `sklearn.model_selection` class, split your data into train and test datasets. Use a test size of 0.33 and a random state of 1. You should expect the output of shapes for `X_train.shape, X_test.shape, y_train.shape, y_test.shape` to be `(339, 13) (167, 13) (339,) (167,)` respectively.

You will later learn that this is a regression predictive modeling problem, meaning that we will be predicting a numeric value. All input variables are also numeric. In this case, we will fit a linear regression algorithm and evaluate model performance by training the model on the training dataset and making a prediction on the test data and evaluate the predictions using the mean absolute error (MAE).

An example of evaluating a linear regression model on the dataset is listed below.

```python
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
...
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)
```

- Using the example above, evaluate the MAE of your data assuming a linear regression model. (Answer: 3.417)
  
  Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

Next, we can try removing outliers from the training dataset. The expectation is that the outliers are causing the linear regression model to learn a bias or skewed understanding of the problem, and that removing these outliers from the training set will allow a more effective model to be learned. We can achieve this by defining the `LocalOutlierFactor` model and using it to make a prediction on the training dataset, marking each row in the training dataset as normal (1) or an outlier (-1). We will use the default hyperparameters for the outlier detection model, although it is a good idea to tune the configuration to the specifics of your dataset.

```python
'''Example of identifying outliers automatically.'''
from sklearn.neighbors import LocalOutlierFactor
...
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
```

We can then use these predictions to remove all outliers from the training dataset.

```python
'''Example of removing identified outliers from the dataset.'''
...
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
```

- With the outlier removed, re-evaluate your model on the regression dataset from the training dataset. (Answer: 3.356)

  Note: Your specific results may vary given the stochastic nature of the learning algorithm, the evaluation procedure, or differences in numerical precision. Consider running the example a few times and compare the average performance.

Firstly, we can see that the number of examples in the training dataset has been reduced from 339 to 305, meaning 34 rows containing outliers were identified and deleted. We can also see a reduction in MAE from about 3.417 by a model fit on the entire training dataset, to about 3.356 on a model fit on the dataset with outliers removed.

The Scikit-Learn library provides other outlier detection algorithms that can be used in the same way such as the Isolation Forest algorithm.

---

## 7. Further Reading

- Outlier on Wikipedia
  https://en.wikipedia.org/wiki/Outlier
  
- Anomaly detection on Wikipedia.
  https://en.wikipedia.org/wiki/Anomaly_detection

- 68-95-99.7 rule on Wikipedia.
  https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule

- Interquartile range.
  https://en.wikipedia.org/wiki/Interquartile_range

- Box plot on Wikipedia.
  https://en.wikipedia.org/wiki/Box_plot